In [1]:
from nlpsandboxclient import client


In [2]:
#data_node_host = "http://data-node-nginx:8080/api/v1"
data_node_host = "http://localhost:8081/api/v1"
annotator_host = "http://localhost:8080/api/v1"

In [ ]:
datasets = client.list_datasets(host=data_node_host)

for dataset in datasets:
      print(dataset)

In [4]:
# select dataset for tests
DATA= "i2b2-test-dataset"
ANNOTATION = "goldstandard"
STORE = "evaluation"
ANNO_TYPES = ["textDateAnnotations", "textPersonNameAnnotations","textLocationAnnotations",
              "textIdAnnotations","textContactAnnotations","textCovidSymptomAnnotations"]
TYPE_DICT = {"textDateAnnotations":"date", "textPersonNameAnnotations":"person","textLocationAnnotations":"location",
              "textIdAnnotations":"id","textContactAnnotations":"contact","textCovidSymptomAnnotations":"covid"}

ANNO_TYPES = ["textCovidSymptomAnnotations"]
TYPE_DICT = {"textCovidSymptomAnnotations":"covid"}

In [5]:
# check if there are annotation duplicates in goldstandards

In [6]:
# need to reload the annotation due to the sandboxclient setting
dataset_id = DATA
annotation_store_id = ANNOTATION
gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=dataset_id,
                                   annotation_store_id=annotation_store_id)
count_dict = {"person":0, "date":0,"contact":0, "id":0,"location":0, "covid":0}

for gs in gs_annos:
    for typ in ANNO_TYPES:
        next_start_position = -1
        same_start_position = -1
        for i in sorted(gs[typ], key=lambda x: x['start']):
            if i["start"] == same_start_position:
                count_dict[TYPE_DICT[typ]] = count_dict[TYPE_DICT[typ]] + 1
                # uncomment this to see the wrong annotations
                
                print(prev, "\n")
                print(i, "\n")
                print(gs["name"],"\n")
                print("\n")
                
            prev = i
            same_start_position = i["start"]
if sum(count_dict.values()) == 0:
    print(f"Test 1 passed on {DATA}, no annotation duplicates")
else:
    print(f"Test 1 failed on {DATA}! Number of duplicated annotations(annotation at the same location): " + str(sum(count_dict.values())))
    print(count_dict)
    

    

Test 1 passed on i2b2-test-dataset, no annotation duplicates


In [17]:
# check if there are cases when instances are split into pieces. 
# e.g. "Dan Smiths" should be one annotation, it's a data quality issue if the "Dan Smiths" is treated as two
# annotations "Dan" and "Smiths"

In [9]:
notes = client.list_notes(host=data_node_host,
                  dataset_id=DATA,
                  fhir_store_id=STORE)

In [10]:
# select dataset for tests

gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=DATA,
                                   annotation_store_id=ANNOTATION)
notes = client.list_notes(host=data_node_host,
                  dataset_id=DATA,
                  fhir_store_id=STORE)
count_dict = {"person":0, "date":0,"contact":0, "id":0,"location":0, "covid":0}
for note, gs in zip(notes, gs_annos):
    
    for typ in ["textPersonNameAnnotations"]:
        next_start_position = -1
        same_start_position = -1
        for i in sorted(gs[typ], key=lambda x: x['start']):
            # if i["start"] == next_start_position and next_start_position-1 >=0 and note["text"][next_start_position-1]  == " " :
            if i["start"] <= next_start_position:    
                if (typ == "textLocationAnnotations" and i["locationType"] == prev["locationType"]) or \
                (typ == "textContactAnnotations" and i["contactType"] == prev["contactType"]) or \
                (typ == "textIdAnnotations" and i["idType"] == prev["idType"]) or \
                typ == "textPersonNameAnnotations"or \
                typ == "textDateAnnotations":
                # uncomment this to see the wrong annotations
                    
                    print(prev, "\n")
                    print(i, "\n")
                    print(gs["name"],"\n")
                    print("\n")
                    #print(note['text'])
                    #if note['text'][(i["start"]+i["length"])] == " ":
                        #print("yes")
                    #print(note['text'][(i["start"]-20):i["start"]+20])
                    #print("\n")
                    count_dict[TYPE_DICT[typ]] = count_dict[TYPE_DICT[typ]] + 1
            prev = i
            next_start_position = i["start"]+i["length"]+1
            
if sum(count_dict.values()) == 0:
    print(f"Test 2 passed on {DATA}, no split annotation")
else:
    print(f"Test 2 failed on {DATA}! Number of split annotations(one annotation split into multiple pieces): " + str(sum(count_dict.values())))
    print(count_dict)

    

Test 2 passed on mayo-covid-dataset, no split annotation


In [20]:
# check if there are annotations that overlap each other.

In [11]:
# select dataset for tests
dataset_id = DATA
annotation_store_id = ANNOTATION
gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=DATA,
                                   annotation_store_id=ANNOTATION)

count_dict = {"person":0, "date":0,"contact":0, "id":0,"location":0, "covid":0}
for gs in gs_annos:
    for typ in ANNO_TYPES:
        next_start_position = -1
        same_start_position = -1
        for i in sorted(gs[typ], key=lambda x: x['start']):
            if i["start"] < next_start_position and i["start"] > same_start_position :
                # uncomment this to see the wrong annotations
                '''
                print(prev, "\n")
                print(i, "\n")
                print(gs["name"],"\n")
                print("\n")
                '''
                count_dict[TYPE_DICT[typ]] = count_dict[TYPE_DICT[typ]] + 1
            prev = i
            next_start_position = i["start"]+i["length"]+1
            same_start_position = i["start"]
if sum(count_dict.values()) == 0:
    print(f"Test 3 passed on {DATA}, no overlapping annotation")
else:
    print(f"Test 3 failed on {DATA}! Number of overlapping annotations: " + str(sum(count_dict.values())))
    print(count_dict)
    

Test 3 passed on mayo-covid-dataset, no overlapping annotation


In [12]:
ANNO_TYPES

['textCovidSymptomAnnotations']

In [ ]:
prob_notes = set()
notes = client.list_notes(host=data_node_host,
                  dataset_id=DATA,
                  fhir_store_id=STORE)

gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=DATA,
                                   annotation_store_id=ANNOTATION)
count_dict = {"person":0, "date":0,"contact":0, "id":0,"location":0, "covid":0}
for note, gs in zip(notes, gs_annos):
    # check if note_id matches
    if gs["name"].split("/")[-1] != note["identifier"]:
        raise Exception("annotation content doesn't match the text")
    for typ in ANNO_TYPES:
        for i in sorted(gs[typ], key=lambda x: x['start']):
            start = i["start"]
            end = i["start"]+i["length"]
            text = i["text"]
            ori_text = note["text"][start:end]
            # this is for uw
            #ori_text = note["text"].replace('\n', '\r\n')[start:end]
            if text != ori_text:
                # uncomment this to see the wrong cases
                
                print("text in annotations: ",text)
                print("real text: ",ori_text)
                print("start position: ",start)
                #print(i["length"])
                #print(len(text),len(ori_text))
                print("noteId", note["identifier"])
                print( "\n")
                
                prob_notes.add(note["identifier"][:-3])
                count_dict[TYPE_DICT[typ]] = count_dict[TYPE_DICT[typ]] + 1
if sum(count_dict.values()) == 0:
    print(f"Test 4 passed on {DATA}, Annotations match the true text")
else:
    print(f"Test 4 failed on {DATA}! Number of annotations that don't match the true text: " + str(sum(count_dict.values())))
    print(count_dict)
    
    

In [ ]:
## COVID SYMPTOM CHECK
dataset_id = DATA
annotation_store_id = ANNOTATION
gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=dataset_id,
                                   annotation_store_id=annotation_store_id)

condition_set = set()
ANNO_TYPES
for gs in gs_annos:
    for typ in ANNO_TYPES:
        for i in gs['textCovidSymptomAnnotations']:
            print(i["text"])
            condition_set.add(i["condition"])
        #print(typ)

In [ ]:
DATA= "i2b2-test-dataset"
ANNOTATION = "goldstandard"
STORE = "evaluation"
ANNO_TYPES = ["textDateAnnotations", "textPersonNameAnnotations","textLocationAnnotations",
              "textIdAnnotations","textContactAnnotations","textCovidSymptomAnnotations"]

ANNO_TYPES = ["textIdAnnotations"]
TYPE_DICT = {"textDateAnnotations":"date", "textPersonNameAnnotations":"person","textLocationAnnotations":"location",
              "textIdAnnotations":"id","textContactAnnotations":"contact","textCovidSymptomAnnotations":"covid"}
dataset_id = DATA
annotation_store_id = ANNOTATION
gs_annos = client.list_annotations(host=data_node_host,
                                   dataset_id=DATA,
                                   annotation_store_id=ANNOTATION)

count_dict = {"person":0, "date":0,"contact":0, "id":0,"location":0, "covid":0}
for gs in gs_annos:
    for typ in ANNO_TYPES:
        print(gs['name'][-6:])
        for i in sorted(gs[typ], key=lambda x: x['start']):
            print(i)
